In [1]:
import os
import pandas as pd
import shutil
import re
import pprint
import math
import numpy as np

# INPUTS

In [2]:
#import all the sheets from the inputs excel file
inputs_sheetDict = pd.read_excel('Inputs.xlsx', sheet_name=None)

In [3]:
# Access individual sheets as dataframes from the dictionary
CUP_inputs_df = inputs_sheetDict['Sheet1']
building_inputs_df = inputs_sheetDict['Sheet2']
TES_inputs_df = inputs_sheetDict['Sheet3']
hotTank_schedule1_inputs_df = inputs_sheetDict['Sheet4']
coldTank_schedule2_inputs_df = inputs_sheetDict['Sheet5']
hotWater_schedule3_inputs_df = inputs_sheetDict['Sheet6']
domesticWater_schedule4_inputs_df = inputs_sheetDict['Sheet7']
chilledCTOcean_schedule5_inputs_df = inputs_sheetDict['Sheet8']
groundReturn_schedule6_inputs_df = inputs_sheetDict['Sheet9']
domesticWater_schedule4_inputs_df.head(15)


,Schedules,Schedules Category,Month,Building
0,Schedules,Domestic Water Temperature,1,55
1,Schedules,Domestic Water Temperature,2,55
2,Schedules,Domestic Water Temperature,3,55
3,Schedules,Domestic Water Temperature,4,55
4,Schedules,Domestic Water Temperature,5,60
5,Schedules,Domestic Water Temperature,6,60
6,Schedules,Domestic Water Temperature,7,60
7,Schedules,Domestic Water Temperature,8,60
8,Schedules,Domestic Water Temperature,9,60
9,Schedules,Domestic Water Temperature,10,60


In [4]:
## Adding formulas for groundReturn_schedule6_inputs_df
# Apply the formula to calculate 'Ht Ex - EWT' column
groundReturn_schedule6_inputs_df['Ht Ex - EWT'] = groundReturn_schedule6_inputs_df['Ht Ex - LWT*'].apply(lambda x: x-10 if not pd.isna(x) else float('nan'))
# Apply the formula to calculate 'Ht Ex - EWT' column
groundReturn_schedule6_inputs_df['Ht Rej - EWT'] = groundReturn_schedule6_inputs_df['Ht Rej - LWT*'].apply(lambda x: 10 + x if not pd.isna(x) else float('nan'))
groundReturn_schedule6_inputs_df.head(15)

,Schedules,Ground Return Temps,Schedules Month,Ht Rej - LWT*,Ht Rej - EWT,Ht Ex - LWT*,Ht Ex - EWT
0,Schedules,Ground Return Temps,1,70,80,60,50
1,Schedules,Ground Return Temps,2,65,75,55,45
2,Schedules,Ground Return Temps,3,60,70,50,40
3,Schedules,Ground Return Temps,4,62,72,52,42
4,Schedules,Ground Return Temps,5,65,75,55,45
5,Schedules,Ground Return Temps,6,70,80,60,50
6,Schedules,Ground Return Temps,7,75,85,65,55
7,Schedules,Ground Return Temps,8,80,90,70,60
8,Schedules,Ground Return Temps,9,84,94,74,64
9,Schedules,Ground Return Temps,10,85,95,75,65


In [5]:
##Adding formulas for Tank_schedule1_inputs_df
# Sum the values of the columns from index 1 to 23 (corresponding to hours 0 to 23)
#hot tank
hotTank_schedule1_inputs_df['Total'] = hotTank_schedule1_inputs_df.loc[:, 0:23].sum(axis=1)
hotTank_schedule1_inputs_df.head(15)
#cold tank
coldTank_schedule2_inputs_df['Total'] = coldTank_schedule2_inputs_df.loc[:, 0:23].sum(axis=1)
coldTank_schedule2_inputs_df.head(15)

,Schedules,Load Shift Control,Month,0,1,2,3,4,5,6,...,15,16,17,18,19,20,21,22,23,Total
0,Schedules,Load Shift Control - Cold Tank by month,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
1,Schedules,Load Shift Control - Cold Tank by month,2,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
2,Schedules,Load Shift Control - Cold Tank by month,3,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
3,Schedules,Load Shift Control - Cold Tank by month,4,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
4,Schedules,Load Shift Control - Cold Tank by month,5,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
5,Schedules,Load Shift Control - Cold Tank by month,6,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
6,Schedules,Load Shift Control - Cold Tank by month,7,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
7,Schedules,Load Shift Control - Cold Tank by month,8,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
8,Schedules,Load Shift Control - Cold Tank by month,9,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9
9,Schedules,Load Shift Control - Cold Tank by month,10,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,9


In [6]:
# the function to changes input dataframe to dictionaries(single value input)

def createDict(df):
    
    dictName = df.iloc[:,0].unique()
    
    # Initialize an empty dictionary to store the data
    data_dict = {}
    
    # Filter out rows with NaN category (if any)
    df = df.dropna(subset=[df.columns[1]])
    
    # Group by category and create nested dictionaries using pandas groupby
    grouped = df.groupby(df.columns[1])
    
    for category, group_df in grouped:
        data_dict[category] = dict(zip(group_df.iloc[:, 2], group_df.iloc[:, 3]))
        
    
    return data_dict  
        
#single input value dicts       
CUP_inputs = createDict(CUP_inputs_df)   
building_inputs = createDict(building_inputs_df) 
TES_inputs = createDict(TES_inputs_df) 
domesticWater_sch4_inputs = createDict(domesticWater_schedule4_inputs_df)
chilledCTOcean_sch5_inputs = createDict(chilledCTOcean_schedule5_inputs_df)

pprint.pprint(building_inputs)

{'Chilled Water': {'Delta T @ Max Load': 16,
                   'Delta T @ Min Load': 8,
                   'Max Load': 45654594.988042854},
 'Domestic Hot Water': {'Load @ Min Approach': 1851684.2228671566,
                        'Max Approach': 20,
                        'Max Load': 7121862.39564291,
                        'Min Approach': 5},
 'Heating Hot Water': {'Delta T @ Max Load': 30,
                       'Delta T @ Min Load': 25,
                       'Max Load': 24679937.927443266},
 'Return HHW Temps given Supply Temp': {110: 15, 130: 20, 160: 30, 180: 40}}


In [7]:
# the function to changes input dataframe to dictionaries(multiple value input)

def createDict_multipleValues(df):
    
  
    # Initialize an empty dictionary to store the data
    data_dict = {}
    
    # Filter out rows with NaN category (if any)
    df = df.dropna(subset=[df.columns[1]])
    
    # Group by category and create nested dictionaries using pandas groupby
    grouped = df.groupby(df.columns[1])
    
    for category, group_df in grouped:
        # Initialize a nested dictionary for the current category
        category_dict = {}
        
        for index,row in group_df.iloc[:,2:].iterrows():
            month = row[ group_df.columns[2]]
            if month not in category_dict:
                     category_dict[month] = {}
            for hour_column,value in row.items():
                # Skip if the column name is the month column or NaN value
                if hour_column == group_df.columns[2]:
                    continue
                
                # Extract the hour (column name as integer)
                hour = hour_column
                
#                 # Create a compound key using month and hour
#                 key = (month, hour)
                
                category_dict[month][hour] = value
#                 print(key)
    # Add the category dictionary to the main data dictionary
    data_dict[category] = category_dict
    return data_dict  

hotTank_sch1_inputs = createDict_multipleValues(hotTank_schedule1_inputs_df)
coldTank_sch2_inputs = createDict_multipleValues(coldTank_schedule2_inputs_df)
hotWater_sch3_inputs = createDict_multipleValues(hotWater_schedule3_inputs_df)
groundReturn_sch6_inputs = createDict_multipleValues(groundReturn_schedule6_inputs_df)
pprint.pprint(hotWater_sch3_inputs)

{'Hot Water Setpoint': {1: {'Building': 130, 'CUP': 170},
                        2: {'Building': 130, 'CUP': 170},
                        3: {'Building': 130, 'CUP': 170},
                        4: {'Building': 130, 'CUP': 170},
                        5: {'Building': 130, 'CUP': 170},
                        6: {'Building': 130, 'CUP': 170},
                        7: {'Building': 130, 'CUP': 170},
                        8: {'Building': 130, 'CUP': 170},
                        9: {'Building': 130, 'CUP': 170},
                        10: {'Building': 130, 'CUP': 170},
                        11: {'Building': 130, 'CUP': 170},
                        12: {'Building': 130, 'CUP': 170}}}


# STATIC INPUTS

In [8]:
#import Weather data 
weather_df = pd.read_excel('Weather data.xlsx')
weather_df.head()

,\n\n\nDate,HH:MM,Day,Month,Dry Bulb Temperature {C},Dew Point Temperature {C},Relative Humidity {%},Dry Bulb Temp (°F),Dew Point Temp (°F),Wet Bulb Temp (°F)
0,2005-01-01,01:00:00,1,1,12.3,8.2,76,54.14,46.76,49.501190
1,2005-01-01,02:00:00,1,1,13.2,7.3,68,55.76,45.14,49.524185
2,2005-01-01,03:00:00,1,1,12.2,8.2,77,53.96,46.76,49.515631
3,2005-01-01,04:00:00,1,1,12.2,7.7,74,53.96,45.86,48.960694
4,2005-01-01,05:00:00,1,1,12.2,7.3,72,53.96,45.14,48.592221


In [9]:
#import Ocean data 
ocean_df = pd.read_excel('Ocean temp.xlsx')
ocean_df.head()

,\n\n\nYear,Month,Day,Surface Temp (°C),Surface Temp (°F)
0,2021,1,1,NaN,55.94
1,2021,1,2,13.3,55.94
2,2021,1,3,13.3,55.94
3,2021,1,4,13.3,55.94
4,2021,1,5,13.4,56.12


In [10]:
#import heat pump data 
heatPump_df = pd.read_excel('CO2 heat pump details.xlsx')
heatPump_df.head()

,CO2 Heat Pump\nLeaving Hot Water (°F),Entering Hot Water (°F),Max Heating Output (Btu/h),HW Flow Rate (gpm),Entering Cold Water (°F),Leaving Cold Water (°F),Max Cooling Output (Btu/h),CHW Flow Rate (gpm),Electrical Input,Cooling COP,Heating COP,Combined COP
0,170,80,146.0,38.9,65,44,104.9,167.9,119.8,3.1,4.3,7.4
1,180,80,146.0,35.0,65,44,104.9,167.9,119.8,3.1,4.3,7.4
2,170,80,142.7,38.1,56,44,101.5,203.0,122.9,2.9,4.1,7.0
3,180,80,142.7,34.3,56,44,101.5,203.0,122.9,2.9,4.1,7.0
4,170,80,140.1,37.4,46,44,98.8,296.5,125.4,2.8,3.9,6.7


In [26]:
#import Building Meta Data
buildingMetaData_df = pd.read_excel('Building_MetaData.xlsx')
buildingMetaData_df.head()

,CAAN_Number,Building Name,Building Category,Area (GSF),Currently on District System (Y/N),Existing Building (Y/N),First Year (yr),Last Year (yr),First Year on loop
0,8000,Banana,Academic,65000,Y,Y,1900,3000,2024
1,8001,Apple,Admin,85000,N,Y,1900,3000,2050
2,8002,Orange,Academic,20000,Y,Y,1900,3000,2024


In [11]:
#import building usage data
buildingUsage_df = pd.read_excel('Building usage data.xlsx')
# Sum the loads
totalSpaceHeating = buildingUsage_df['Heating Load (Btu/h)'].sum()
totalDHW = buildingUsage_df['DHW Load (Btu/h)'].sum()
totalHeating = buildingUsage_df['Total Heating Load (Btu/h)'].sum()
totalLoad = buildingUsage_df['Total Load (Btu/h)'].sum()
buildingUsage_df.head()



,Date and Time,Heating Load (Btu/h),DHW Load (Btu/h),Cooling Load (Btu/h),Total Heating Load (Btu/h),Total Load (Btu/h)
0,2010-01-01 00:00:00,6.158529e+06,1.913677e+06,48.929039,8.072206e+06,8.072255e+06
1,2010-01-01 01:00:00,4.513664e+06,1.618239e+06,349.285328,6.131903e+06,6.132252e+06
2,2010-01-01 02:00:00,5.149001e+06,1.734384e+06,349.285328,6.883384e+06,6.883734e+06
3,2010-01-01 03:00:00,5.668696e+06,1.582614e+06,349.285328,7.251309e+06,7.251658e+06
4,2010-01-01 04:00:00,5.888233e+06,2.090599e+06,349.285328,7.978832e+06,7.979181e+06


In [12]:
#calculating the inputs
# CUP setpoints
maxValue_hotWater = hotWater_schedule3_inputs_df["CUP"].max()
CUP_inputs["Setpoints"]["Hot Water Supply"] = maxValue_hotWater
maxValue_chilledWater = chilledCTOcean_schedule5_inputs_df.loc[chilledCTOcean_schedule5_inputs_df["Schedules Category"]=="Chilled Water Setpoint","CUP"].max()
CUP_inputs["Setpoints"]["Chilled Water Supply"] = maxValue_chilledWater

#CUP electric chillers
CUP_inputs["Electric Chillers"]["Capacity for Min Lift"] = CUP_inputs["Electric Chillers"]["Max Capacity"]*0.3

#building Chilled water
maxValue_cooling = buildingUsage_df['Cooling Load (Btu/h)'].max()
building_inputs["Chilled Water"]["Max Load"]= maxValue_cooling

#building DHW
maxValue_DHW = buildingUsage_df['DHW Load (Btu/h)'].max()
building_inputs["Domestic Hot Water"]["Max Load"]= maxValue_DHW
building_inputs["Domestic Hot Water"]["Load @ Min Approach"]= maxValue_DHW*0.26

#building HHW
maxValue_HHW = buildingUsage_df['Heating Load (Btu/h)'].max()
building_inputs["Heating Hot Water"]["Max Load"]= maxValue_HHW + building_inputs["Domestic Hot Water"]["Max Load"]

#TES Tank Tank Properties
CUFtoGAL = TES_inputs["Conversion Rates"]["CUF to GAL"]
hot_radius = TES_inputs["Tank Properties"]["TES Hot Diameter"]/2
hot_height = TES_inputs["Tank Properties"]["TES Hot Height"]
TES_inputs["Tank Properties"]["TES Hot Tank Volume"]= math.pi * (hot_radius ** 2) * hot_height * CUFtoGAL
cold_radius = TES_inputs["Tank Properties"]["TES Cold Diameter"]/2
cold_height = TES_inputs["Tank Properties"]["TES Cold Height"]
TES_inputs["Tank Properties"]["TES Cold Tank Volume"]= math.pi * (cold_radius ** 2) * cold_height * CUFtoGAL

#TES Initial Conditions
# TES Hot Tank Upper Tank Volume ----- Check the condition and calculate the result accordingly
if TES_inputs["Tank Properties"]["TES Hot Tank Volume"] == 0:
    result = 0
else:
    hot_volume1 = TES_inputs["Tank Properties"]["TES Hot Tank Volume"]
    hot_volume2 =TES_inputs["Initial Conditions"]["TES Hot Thermocline thickness"] * math.pi * (hot_radius ** 2)* CUFtoGAL   # Volume of second part of the cylinder
    difference = hot_volume1 - hot_volume2           # Difference between the volumes
    TES_inputs["Initial Conditions"]["TES Hot Tank Upper Tank Volume"] = difference * TES_inputs["Initial Conditions"]["Starting TES Hot Charge"]          
TESHotUpperTankVolume = TES_inputs["Initial Conditions"]["TES Hot Tank Upper Tank Volume"]
    
# TES Hot Thermocline volume
if TES_inputs["Tank Properties"]["TES Hot Tank Volume"] == 0:
    result = 0
else:
    TES_inputs["Initial Conditions"]["TES Hot Thermocline volume"] =  hot_volume2

    
#TES Hot Tank Lower Tank Volume    
TES_inputs["Initial Conditions"]["TES Hot Tank Lower Tank Volume"] =  hot_volume1 - hot_volume2 - TESHotUpperTankVolume    
    
# TES Cold Tank Lower Tank Volume ----- Check the condition and calculate the result accordingly
if TES_inputs["Tank Properties"]["TES Cold Tank Volume"] == 0:
    result = 0
else:
    cold_volume1 =TES_inputs["Tank Properties"]["TES Cold Tank Volume"]
    cold_volume2 =TES_inputs["Initial Conditions"]["TES Cold Thermocline thickness"] * math.pi * (cold_radius ** 2)* CUFtoGAL   # Volume of second part of the cylinder
    difference = cold_volume1 - cold_volume2           # Difference between the volumes
    TES_inputs["Initial Conditions"]["TES Cold Tank Lower Tank Volume"] = difference * TES_inputs["Initial Conditions"]["Starting TES Cold Charge"]           
TESColdLowerTankVolume =  TES_inputs["Initial Conditions"]["TES Cold Tank Lower Tank Volume"]
# TES cold Thermocline volume
if TES_inputs["Tank Properties"]["TES Cold Tank Volume"] == 0:
    result = 0
else:
    TES_inputs["Initial Conditions"]["TES Cold Thermocline volume"] =  cold_volume2 
#TES Hot Tank Lower Tank Volume    
TES_inputs["Initial Conditions"]["TES Cold Tank Upper Tank Volume"] =  cold_volume1 - cold_volume2 - TESColdLowerTankVolume

#Tank Charging Characteristics
#TES Hot Capacity
GALtoLbs = TES_inputs["Conversion Rates"]["GAL to Lbs"]
upperHotTemp = TES_inputs["Initial Conditions"]["TES Hot Tank Upper Tank Temp"]
lowerHotTemp = TES_inputs["Initial Conditions"]["TES Hot Tank Lower Tank Temp"]
TES_inputs["Tank Charging Characteristics"]["TES Hot Capacity"] = GALtoLbs*(upperHotTemp-lowerHotTemp)*hot_volume1/1000
#TES Hot Max Flow 
tankVolumeMoved = TES_inputs["Tank Charging Characteristics"]["TES Hot Max % of Total Tank Volume moved in 1 hr"]
TES_inputs["Tank Charging Characteristics"]["TES Hot Max Flow"] = math.ceil((hot_volume1 * tankVolumeMoved) / 60 / 5) * 5
#TES Hot Max Flow 
hot_tankVolumeMoved = TES_inputs["Tank Charging Characteristics"]["TES Hot Max % of Total Tank Volume moved in 1 hr"]
TES_inputs["Tank Charging Characteristics"]["TES Hot Max Flow"] = math.ceil((hot_volume1 * hot_tankVolumeMoved) / 60 / 5) * 5

#TES Cold Capacity
GALtoLbs = TES_inputs["Conversion Rates"]["GAL to Lbs"]
upperColdTemp = TES_inputs["Initial Conditions"]["TES Cold Tank Upper Tank Temp"]
lowerColdTemp = TES_inputs["Initial Conditions"]["TES Cold Tank Lower Tank Temp"]
TES_inputs["Tank Charging Characteristics"]["TES Cold Capacity"] = GALtoLbs*(upperColdTemp-lowerColdTemp)*cold_volume1/1000
#TES Cold Max Flow 
cold_tankVolumeMoved = TES_inputs["Tank Charging Characteristics"]["TES Cold Max % of Total Tank Volume moved in 1 hr"]
TES_inputs["Tank Charging Characteristics"]["TES Cold Max Flow"] = math.ceil((cold_volume1 * cold_tankVolumeMoved) / 60 / 5) * 5




# Building heating hot water 
pprint.pprint(TES_inputs)


{'Conversion Rates': {'CUF to GAL': 7.48, 'GAL to Lbs': 8.34},
 'Initial Conditions': {'Starting TES Cold Charge': 0.99,
                        'Starting TES Hot Charge': 0.99,
                        'TES Cold Tank Lower Tank Temp': 44.0,
                        'TES Cold Tank Lower Tank Volume': 1214387.1641385558,
                        'TES Cold Tank Upper Tank Temp': 60.0,
                        'TES Cold Tank Upper Tank Volume': 12266.53701150068,
                        'TES Cold Thermocline thickness': 2.0,
                        'TES Cold Thermocline volume': 42298.40348793298,
                        'TES Hot Tank Lower Tank Temp': 80.0,
                        'TES Hot Tank Lower Tank Volume': 12266.53701150068,
                        'TES Hot Tank Upper Tank Temp': 170.0,
                        'TES Hot Tank Upper Tank Volume': 1214387.1641385558,
                        'TES Hot Thermocline thickness': 2.0,
                        'TES Hot Thermocline volume': 42298.

# Building module inputs (same for all bldgs)

In [13]:
buildingModule_inputs =  pd.DataFrame()

In [14]:
buildingModule_inputs["Ambient Temperature (°F)"] = weather_df["Dry Bulb Temp (°F)"]
buildingModule_inputs["Ambient Temperature (°F)"] = weather_df["Dry Bulb Temp (°F)"]

##Loop HW STP (°F)
# Extract columns from input_fields_df
hotWaterMonths = hotWater_schedule3_inputs_df["Schedules Month"].values
hotWaterSetpointCUP = hotWater_schedule3_inputs_df["CUP"].values


# Extract month from building_module_df
buildingUsage_df['Date and Time'] = pd.to_datetime(buildingUsage_df['Date and Time'], unit='D')
month_to_match = buildingUsage_df['Date and Time'].dt.month.values

# Find the index where month matches
HW_indices = np.where(month_to_match[:, None] == hotWaterMonths )[1]

# Get the corresponding value from hotWaterSetpoint column
buildingModule_inputs["Loop HW STP (°F)"]  = hotWaterSetpointCUP[HW_indices]

##Building HHW STP (°F)
# Extract columns from input_fields_df
hotWaterSetpointBldg = hotWater_schedule3_inputs_df["Building"].values

# Get the corresponding value from hotWaterSetpoint column
buildingModule_inputs["Building HHW STP (°F)"]  = hotWaterSetpointBldg[HW_indices]


##Loop CHW STP (°F)
# Extract columns from input_fields_df
chilledWaterMonths = chilledCTOcean_schedule5_inputs_df.loc[chilledCTOcean_schedule5_inputs_df["Schedules Category"]=="Chilled Water Setpoint","Month"].values
chilledWaterSetpointBldg =chilledCTOcean_schedule5_inputs_df.loc[chilledCTOcean_schedule5_inputs_df["Schedules Category"]=="Chilled Water Setpoint","CUP"].values

# Find the index where month matches
CH_indices = np.where(month_to_match[:, None] == chilledWaterMonths)[1]

# Get the corresponding value from hotWaterSetpoint column
buildingModule_inputs["Loop CHW STP (°F)"]  = chilledWaterSetpointBldg[CH_indices]


buildingModule_inputs.head(15)

,Ambient Temperature (°F),Loop HW STP (°F),Building HHW STP (°F),Loop CHW STP (°F)
0,54.14,170,130,44.0
1,55.76,170,130,44.0
2,53.96,170,130,44.0
3,53.96,170,130,44.0
4,53.96,170,130,44.0
5,53.96,170,130,44.0
6,54.14,170,130,44.0
7,55.22,170,130,44.0
8,56.30,170,130,44.0
9,59.18,170,130,44.0


# Building module outputs

### To be looped through every building

In [15]:
#defining inputs ---> static-same for all buildings
HW_LoopSTP=buildingModule_inputs["Loop HW STP (°F)"]
HW_supplyLosses=CUP_inputs["Setpoints"]["Hot Water Supply Losses"]
CHW_LoopSTP=buildingModule_inputs["Loop CHW STP (°F)"]
CHW_supplyLosses=CUP_inputs["Setpoints"]["Chilled Water Supply Losses"]
CHW_deltaT_Max=building_inputs["Chilled Water"]["Delta T @ Max Load"]
CHW_deltaT_Min=building_inputs["Chilled Water"]["Delta T @ Min Load"]
CHW_maxLoad= building_inputs["Chilled Water"]["Max Load"]
HHW_BldgSTP= buildingModule_inputs["Building HHW STP (°F)"]
HHW_supply_Temps = building_inputs_df.loc[building_inputs_df["Building Level Categories"]=="Return HHW Temps given Supply Temp","Building Level User Inputs"]
HHW_return_Temps = building_inputs_df.loc[building_inputs_df["Building Level Categories"]=="Return HHW Temps given Supply Temp","Building Level Values"]
DHWMonths = domesticWater_schedule4_inputs_df["Month"]
DHWSetpoint = domesticWater_schedule4_inputs_df["Building"]
DHWmaxLoad = building_inputs["Domestic Hot Water"]["Max Load"]
DHWmaxApproach = building_inputs["Domestic Hot Water"]["Max Approach"]
DHWminApproach = building_inputs["Domestic Hot Water"]["Min Approach"]
DHWloadMinApproach = building_inputs["Domestic Hot Water"]["Load @ Min Approach"]

In [16]:
# Reset the index to be sequential (0, 1, 2, 3, ...)
HHW_return_Temps = HHW_return_Temps.reset_index(drop=True)
# data type set to int
HHW_supply_Temps = HHW_supply_Temps.astype(int)

In [17]:
# CHW_maxLoad_int = int(CHW_maxLoad)
# DHWmaxLoad_int = int(DHWmaxLoad)
# DHWloadMinApproach_int = int(DHWloadMinApproach)
# print(CHW_maxLoad_int)

In [18]:

from ucsbdistrict.building_staticInputs import BuildingParameters
#creating instance by assigning static parameters
params_bldg_1 = BuildingParameters(HW_LoopSTP=HW_LoopSTP, HW_supplyLosses=HW_supplyLosses,CHW_LoopSTP=CHW_LoopSTP,\
                                  CHW_supplyLosses=CHW_supplyLosses,CHW_deltaT_Max=CHW_deltaT_Max, CHW_deltaT_Min=CHW_deltaT_Min,\
                                  CHW_maxLoad=CHW_maxLoad,HHW_supply_Temps=HHW_supply_Temps,HHW_return_Temps=HHW_return_Temps,HHW_BldgSTP=HHW_BldgSTP,\
                                  DHWMonths=DHWMonths,DHWSetpoint=DHWSetpoint,buildingDate=buildingUsage_df['Date and Time'],hotWaterMonths=hotWaterMonths,\
                                  DHWmaxLoad=DHWmaxLoad,DHWmaxApproach=DHWmaxApproach,DHWminApproach=DHWminApproach,DHWloadMinApproach=DHWloadMinApproach)


In [27]:
#filtered district building meta data
districtMetaData_df = buildingMetaData_df[buildingMetaData_df["Currently on District System (Y/N)"]=="Y"]

districtMetaData_df.head()

,CAAN_Number,Building Name,Building Category,Area (GSF),Currently on District System (Y/N),Existing Building (Y/N),First Year (yr),Last Year (yr),First Year on loop
0,8000,Banana,Academic,65000,Y,Y,1900,3000,2024
2,8002,Orange,Academic,20000,Y,Y,1900,3000,2024


In [36]:
from ucsbdistrict.building_module import Building
#path to folder that contains all building calibrated data files
building_usage_folder =  r"C:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\Building usage data"
# Get a list of all files in the folder
files = os.listdir(building_usage_folder)
allBldgModule_output =pd.DataFrame()
#loop through all files
for file in files:
    print(file)
    file_path = os.path.join(building_usage_folder, file)
    buildingMockUsage_df = pd.read_excel(file_path)
    if buildingMockUsage_df['CAAN_Number'][0] in districtMetaData_df['CAAN_Number'].values:
        #creating building instance bu  assinging changing / calculated parameters(fields) and calculating outputs
        bldg = Building(parameters=params_bldg_1, caan_no=buildingMockUsage_df['CAAN_Number'][0], name="Lab",coolingLoad = buildingMockUsage_df['Cooling Load (Btu/h)'],\
                        heatingLoad =buildingMockUsage_df['Heating Load (Btu/h)'],DHWLoad =buildingMockUsage_df['DHW Load (Btu/h)'],\
                        timeStamp = buildingMockUsage_df['Date and Time'])
        buildingModule_output = bldg.compute()
        allBldgModule_output = pd.concat([allBldgModule_output, buildingModule_output], ignore_index=True)  # Concatenate new row to existing DataFrame

allBldgModule_output.head(10)

8000.xlsx
8001.xlsx
8002.xlsx


,caan_no,Time Stamp,Loop HWST @ Building (°F),Loop CHWST @ Building (°F),CHWRT (°F),CHWR Flow (gpm),Building HHWRT (°F),HHWRflow,Building Domestic Water Temp (°F),Building DHWRT (°F),Building DHWR Flow (gpm),District HWS Flow (gpm),Bypassed Return to HHWS (gpm),District HWR Flow (gpm),District HWRT (°F),Check Building HWS = HWR
0,8000,2010-01-01 00:00:00,165,46.0,54.000009,0.012232,110,615.852893,55,60.176443,36.512341,260.458848,391.906386,260.458848,103.0155,True
1,8000,2010-01-01 01:00:00,165,46.0,54.000061,0.087321,110,451.366397,55,60.000000,30.823601,194.956836,287.233162,194.956836,102.094763,True
2,8000,2010-01-01 02:00:00,165,46.0,54.000061,0.087321,110,514.90006,55,60.000000,33.035878,220.272264,327.663675,220.272264,102.501126,True
3,8000,2010-01-01 03:00:00,165,46.0,54.000061,0.087321,110,566.869554,55,60.000000,30.145019,236.279403,360.735171,236.279403,103.620896,True
4,8000,2010-01-01 04:00:00,165,46.0,54.000061,0.087321,110,588.82329,55,60.679999,40.080497,254.198057,374.70573,254.198057,102.223504,True
5,8000,2010-01-01 05:00:00,165,46.0,54.000070,0.099553,110,511.227764,55,62.597476,53.988803,239.889808,325.326759,239.889808,99.331745,True
6,8000,2010-01-01 06:00:00,165,46.0,54.000078,0.111785,110,475.412885,55,66.261078,82.064580,254.941993,302.535472,254.941993,95.920655,True
7,8000,2010-01-01 07:00:00,165,46.0,54.000087,0.124017,110,416.342578,55,69.599058,109.522346,260.919647,264.945277,260.919647,93.0415,True
8,8000,2010-01-01 08:00:00,165,46.0,54.000026,0.036697,110,364.268861,55,71.699312,127.805731,260.267135,231.807457,260.267135,91.192218,True
9,8000,2010-01-01 09:00:00,165,46.0,54.189233,263.740986,110,546.284975,55,73.478893,143.954306,342.603388,347.635893,342.603388,94.654645,True


In [39]:
# Convert 'Time Stamp' column to datetime format
allBldgModule_output['Time Stamp'] = pd.to_datetime(allBldgModule_output['Time Stamp'])

# Group by 'Time Stamp' and sum the 'Loop HWST @ Building (°F)' column
sum_by_timestamp = allBldgModule_output.groupby('Time Stamp')['Loop HWST @ Building (°F)'].sum()
print(sum_by_timestamp.shape)

(8760,)


In [35]:
print(8760*2)

17520


In [21]:
from ucsbdistrict.building_module import Building
#creating building instance bu  assinging changing / calculated parameters(fields) and calculating outputs
bldg_1 = Building(parameters=params_bldg_1, caan_no=1000, name="Lab",coolingLoad = buildingUsage_df['Cooling Load (Btu/h)'],\
                 heatingLoad =buildingUsage_df['Heating Load (Btu/h)'],DHWLoad =buildingUsage_df['DHW Load (Btu/h)'],\
                 timeStamp = buildingMockUsage_df['Date and Time'])
buildingModule_output = bldg_1.compute()
buildingModule_output.head(15)

,caan_no,Time Stamp,Loop HWST @ Building (°F),Loop CHWST @ Building (°F),CHWRT (°F),CHWR Flow (gpm),Building HHWRT (°F),HHWRflow,Building Domestic Water Temp (°F),Building DHWRT (°F),Building DHWR Flow (gpm),District HWS Flow (gpm),Bypassed Return to HHWS (gpm),District HWR Flow (gpm),District HWRT (°F),Check Building HWS = HWR
0,1000,2010-01-01 00:00:00,165,46.0,54.000009,0.012232,110,615.852893,55,60.176443,36.512341,260.458848,391.906386,260.458848,103.0155,True
1,1000,2010-01-01 01:00:00,165,46.0,54.000061,0.087321,110,451.366397,55,60.000000,30.823601,194.956836,287.233162,194.956836,102.094763,True
2,1000,2010-01-01 02:00:00,165,46.0,54.000061,0.087321,110,514.90006,55,60.000000,33.035878,220.272264,327.663675,220.272264,102.501126,True
3,1000,2010-01-01 03:00:00,165,46.0,54.000061,0.087321,110,566.869554,55,60.000000,30.145019,236.279403,360.735171,236.279403,103.620896,True
4,1000,2010-01-01 04:00:00,165,46.0,54.000061,0.087321,110,588.82329,55,60.679999,40.080497,254.198057,374.70573,254.198057,102.223504,True
5,1000,2010-01-01 05:00:00,165,46.0,54.000070,0.099553,110,511.227764,55,62.597476,53.988803,239.889808,325.326759,239.889808,99.331745,True
6,1000,2010-01-01 06:00:00,165,46.0,54.000078,0.111785,110,475.412885,55,66.261078,82.064580,254.941993,302.535472,254.941993,95.920655,True
7,1000,2010-01-01 07:00:00,165,46.0,54.000087,0.124017,110,416.342578,55,69.599058,109.522346,260.919647,264.945277,260.919647,93.0415,True
8,1000,2010-01-01 08:00:00,165,46.0,54.000026,0.036697,110,364.268861,55,71.699312,127.805731,260.267135,231.807457,260.267135,91.192218,True
9,1000,2010-01-01 09:00:00,165,46.0,54.189233,263.740986,110,546.284975,55,73.478893,143.954306,342.603388,347.635893,342.603388,94.654645,True
